In [ ]:
# ============================================================
# 1. ENVIRONMENT SETUP
# ============================================================


import sys
import os

class DevNull:
    def write(self, msg):
        pass
    def flush(self):
        pass

sys.stderr = DevNull()

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy.stats import ttest_rel, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import combinations
from scipy.stats import ttest_rel

In [ ]:
# Load dataset
df = pd.read_csv("/kaggle/input/financial-headlines-augmented/news_augmented.csv")
texts = df["headline"].tolist()
labels = df["sentiment"].map({"negative":0, "neutral":1, "positive":2}).values


In [ ]:
df.shape

In [ ]:
# ============================================================
# 2. LOAD CV RESULTS FROM PICKLE + EXTRACT METRICS
# ============================================================

import pickle

bert_raw       = pickle.load(open("/kaggle/input/bert-zeeshan/bert_5fold_cv_results_cached.pkl", "rb"))
distil_raw     = pickle.load(open("/kaggle/input/distilbert-zeeshan/distilbert_5fold_cv_results_cached.pkl", "rb"))
albert_raw     = pickle.load(open("/kaggle/input/zeeshan-albert/albert_5fold_cv_results_cached.pkl", "rb"))
roberta_raw    = pickle.load(open("/kaggle/input/roberta-zeeshan/roberta_5fold_cv_results_cached.pkl", "rb"))
finbert_raw    = pickle.load(open("/kaggle/input/zeeshan-finbert/finbert_5fold_cv_results_cached.pkl", "rb"))

def extract_metrics(folds):
    accuracy  = [f["eval_accuracy"]  for f in folds]
    precision = [f["eval_precision"] for f in folds]
    recall    = [f["eval_recall"]    for f in folds]
    f1        = [f["eval_f1"]        for f in folds]

    return {
        "accuracy_mean":  np.mean(accuracy),
        "precision_mean": np.mean(precision),
        "recall_mean":    np.mean(recall),
        "f1_mean":        np.mean(f1),

        "accuracy_all": accuracy,
        "precision_all": precision,
        "recall_all": recall,
        "f1_all": f1
    }

bert       = extract_metrics(bert_raw)
distilbert = extract_metrics(distil_raw)
albert     = extract_metrics(albert_raw)
roberta    = extract_metrics(roberta_raw)
finbert    = extract_metrics(finbert_raw)

In [ ]:
# ============================================================
# 3. UNIFIED MODEL REGISTRY 
# ============================================================

model_registry = {
    "BERT": {
        "results": bert,
        "path": "/kaggle/input/bert-zeeshan/news_headline_sentiment/checkpoint-282",
        "color": "#ff7f0e"
    },
    "DistilBERT": {
        "results": distilbert,
        "path": "/kaggle/input/distilbert-zeeshan/news_headline_sentiment/checkpoint-376",
        "color": "#2ca02c"
    },
    "ALBERT": {
        "results": albert,
        "path": "/kaggle/input/zeeshan-albert/news_headline_sentiment/checkpoint-376",
        "color": "#9467bd"
    },
    "RoBERTa": {
        "results": roberta,
        "path": "/kaggle/input/roberta-zeeshan/news_headline_sentiment/checkpoint-235",
        "color": "#1f77b4"
    },
    "FinBERT": {
        "results": finbert,
        "path": "/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376",
        "color": "#d62728"
    }
}


In [ ]:
# ============================================================
# 4. BUILD RESULTS DATAFRAME
# ============================================================

results_df = pd.DataFrame({
    model: {
        "Accuracy": info["results"]["accuracy_mean"],
        "Precision": info["results"]["precision_mean"],
        "Recall": info["results"]["recall_mean"],
        "F1": info["results"]["f1_mean"]
    }
    for model, info in model_registry.items()
}).T.round(4)

display(results_df)

In [ ]:
# ============================================================
# 5. VISUALIZATIONS (BAR, RADAR, BOXPLOT, HEATMAP, SCATTER)
# ============================================================

# Accuracy barplot
plt.figure(figsize=(8,5))
sns.barplot(x=results_df.index, y=results_df["Accuracy"], palette="Set2")
plt.title("Model Accuracy Comparison")
plt.ylim(0.80, 1.00)
plt.show()



In [ ]:
# F1 score barplot
plt.figure(figsize=(8,5))
sns.barplot(x=results_df.index, y=results_df["F1"], palette="Set3")
plt.title("Model F1 Score Comparison")
plt.ylim(0.80, 1.00)
plt.show()

In [ ]:
# ============================================================
# Fold-by-Fold Variation of Macro-F1 and Validation Loss
# (All Models in One Plot)
# ============================================================

import matplotlib.pyplot as plt
import numpy as np

models_cv = {
    "BERT": bert_raw,
    "DistilBERT": distil_raw,
    "ALBERT": albert_raw,
    "RoBERTa": roberta_raw,
    "FinBERT": finbert_raw
}

colors = {
    "BERT": "#ff7f0e",
    "DistilBERT": "#2ca02c",
    "ALBERT": "#9467bd",
    "RoBERTa": "#1f77b4",
    "FinBERT": "#d62728"
}

folds = np.arange(1, 6)

fig, ax1 = plt.subplots(figsize=(12, 7))

#  Plot Macro-F1 (Left Axis)
for model, results in models_cv.items():
    f1_scores = [r["eval_f1"] for r in results]
    ax1.plot(
        folds,
        f1_scores,
        marker="o",
        linewidth=2,
        color=colors[model],
        label=f"{model} F1"
    )

ax1.set_xlabel("Cross-Validation Fold")
ax1.set_ylabel("Macro-F1 Score")
ax1.set_xticks(folds)
ax1.set_ylim(0.85, 0.92)
ax1.grid(alpha=0.3)

#  Plot Validation Loss (Right Axis)
ax2 = ax1.twinx()

for model, results in models_cv.items():
    val_losses = [r["eval_loss"] for r in results]
    ax2.plot(
        folds,
        val_losses,
        linestyle="--",
        linewidth=2,
        color=colors[model],
        alpha=0.6,
        label=f"{model} Val Loss"
    )

ax2.set_ylabel("Validation Loss")

#  Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(
    lines1 + lines2,
    labels1 + labels2,
    loc="center left",
    bbox_to_anchor=(1.02, 0.5),
    frameon=True
)

plt.title("Fold-by-Fold Variation of Macro-F1 Score and Validation Loss Across Models")
plt.tight_layout()
plt.show()


In [ ]:
# Radar chart
from math import pi
metrics = ["Accuracy", "Precision", "Recall", "F1"]
N = len(metrics)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

plt.figure(figsize=(8,8))
ax = plt.subplot(111, polar=True)

for model in results_df.index:
    values = results_df.loc[model].tolist() + [results_df.loc[model].tolist()[0]]
    ax.plot(angles, values, linewidth=2, label=model)
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics)
plt.title("Model Performance Radar Chart")
plt.legend(loc="upper right")
plt.show()


In [ ]:
# Boxplot across 5-fold CV
cv_df = pd.DataFrame({
    "Model": sum([[m]*5 for m in model_registry.keys()], []),
    "F1": sum([model_registry[m]["results"]["f1_all"] for m in model_registry], [])
})

plt.figure(figsize=(10,6))
sns.boxplot(x="Model", y="F1", data=cv_df, palette="Set2")
plt.title("F1 Across 5-Fold Cross Validation")
plt.ylim(0.80, 1.00)
plt.show()

In [ ]:
# Heatmap
plt.figure(figsize=(8,6))
sns.heatmap(results_df, annot=True, cmap="YlGnBu", fmt=".3f")
plt.title("Heatmap of Model Performance Metrics")
plt.show()

In [ ]:
# ============================================================
# 6. ROC CURVE (MICRO-AVERAGED)
# ============================================================

def get_micro_roc(model_path, name, color):
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    probs = []
    batch_size = 64
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = tokenizer(batch, padding=True, truncation=True,
                               max_length=128, return_tensors="pt").to(device)
            outputs = model(**inputs)
            prob = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            probs.append(prob)

    y_score = np.vstack(probs)
    y_bin = label_binarize(labels, classes=[0,1,2])

    fpr, tpr, _ = roc_curve(y_bin.ravel(), y_score.ravel())
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, roc_auc

plt.figure(figsize=(9,7.5))

for model, info in model_registry.items():
    fpr, tpr, roc_auc = get_micro_roc(info["path"], model, info["color"])
    plt.plot(fpr, tpr, lw=3.5, color=info["color"],
             label=f"{model} (AUC = {roc_auc:.3f})")

plt.plot([0,1],[0,1],"--",color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# ============================================================
# 7. STATISTICAL TESTS (T-TEST, ANOVA, TUKEY)
# ============================================================

# Collect fold-level F1 scores for each model
f1_scores = {
    "BERT": bert["f1_all"],
    "DistilBERT": distilbert["f1_all"],
    "ALBERT": albert["f1_all"],
    "RoBERTa": roberta["f1_all"],
    "FinBERT": finbert["f1_all"]
}
# Perform paired t-tests for all model pairs
ttest_rows = []

for (model_a, scores_a), (model_b, scores_b) in combinations(f1_scores.items(), 2):
    t_stat, p_value = ttest_rel(scores_a, scores_b)
    mean_diff = np.mean(scores_a) - np.mean(scores_b)

    ttest_rows.append({
        "Model A": model_a,
        "Model B": model_b,
        "Mean F1 (A)": np.mean(scores_a),
        "Mean F1 (B)": np.mean(scores_b),
        "Mean Difference (A − B)": mean_diff,
        "t-statistic": t_stat,
        "p-value": p_value,
        "Significant (α = 0.05)": "Yes" if p_value < 0.05 else "No"
    })

# Convert to DataFrame
ttest_df = pd.DataFrame(ttest_rows).round(5)

display(ttest_df)


In [ ]:
# Cohen’s d effect size
finbert_scores = model_registry["FinBERT"]["results"]["f1_all"]
def cohens_d_paired(x, y):
    diff = np.array(x) - np.array(y)
    return diff.mean() / diff.std(ddof=1)

for model, info in model_registry.items():
    if model != "FinBERT":
        d = cohens_d_paired(finbert_scores, info["results"]["f1_all"])
        print(f"FinBERT vs {model}: Cohen's d = {d:.3f}")

In [ ]:
# ANOVA
anova_stat, anova_p = f_oneway(
    *[info["results"]["f1_all"] for info in model_registry.values()]
)
print("\nANOVA:", anova_stat, anova_p)


In [ ]:
# Tukey HSD
tukey_df = pd.DataFrame({
    "F1": sum([info["results"]["f1_all"] for info in model_registry.values()], []),
    "Model": sum([[m]*5 for m in model_registry.keys()], [])
})
tukey_results = pairwise_tukeyhsd(tukey_df["F1"], tukey_df["Model"])
print(tukey_results)

In [ ]:
# ============================================================
# 8. Tukey CI Plot
# ============================================================

model_order = list(model_registry.keys())

model_means = {m: np.mean(model_registry[m]["results"]["f1_all"]) for m in model_order}

ci_bounds = {}
for m in model_order:
    vals = model_registry[m]["results"]["f1_all"]
    mean = np.mean(vals)
    sem = np.std(vals) / np.sqrt(len(vals))
    ci_bounds[m] = (mean - 1.96*sem, mean + 1.96*sem)

means = [model_means[m] for m in model_order]
lower_ci = [ci_bounds[m][0] for m in model_order]
upper_ci = [ci_bounds[m][1] for m in model_order]

plt.figure(figsize=(10,6))
y = np.arange(len(model_order))

plt.hlines(y, lower_ci, upper_ci, colors="black", linewidth=2)
plt.plot(means, y, "o", color="darkred", markersize=8)
plt.yticks(y, model_order)
plt.xlabel("F1 Score")
plt.title("Tukey HSD 95% Confidence Intervals")
plt.grid(alpha=0.3)
plt.show()